In [1]:
import pulp
import pandas as pd
import init_data

In [2]:
# データの読み込み
S, I, V, P, T, t,b, C, K = init_data.load_data()
b = {key: 1 - value for key, value in b.items()}
# 問題の定義
prob = pulp.LpProblem('karaoke', pulp.LpMaximize)
# 変数の定義
x = pulp.LpVariable.dicts('x', [(i,s,k)for i in I for s in S for k in K], cat='Binary')
y = pulp.LpVariable.dicts('y', [k for k in K], cat='Binary')
# 目的関数の定義
prob += pulp.lpSum([V[(i,s)]*x[(i,s,k)] for i in I for s in S for k in K])*100 + pulp.lpSum([x[(i,s,k)] for i in I for s in C for k in K])*2000 - pulp.lpSum([y[k] for k in K])*1
# 制約条件の定義 
# 1. 1つの番目の人は1回に1曲しか歌えない
for k in K:
    prob += pulp.lpSum([x[(i,s,k)] for i in I for s in S]) <= 1
# 2. 順番が若い順になる
for k in K:
    if k != K[-1]:
        prob += pulp.lpSum([x[(i,s,k)] for i in I for s in S]) >= pulp.lpSum([x[(i,s,k+1)] for i in I for s in S])
# 3.時間内に収める
prob += pulp.lpSum([t[s]*x[(i,s,k)] for i in I for s in S for k in K]) <= T
# # 4. 交互に歌う
for i in I:
    for k in K:
        if k != K[-1]:
            prob += pulp.lpSum([x[(i,s,k)] for s in S]) + pulp.lpSum([x[(i,s,k+1)] for s in S])<=1
# #5. 二人で同じ曲は二回歌えない
for s in S:
    prob += pulp.lpSum([x[(i,s,k)] for i in I for k in K]) <= 1
#6. 共通の場合、歌ってる回数が多いほうが歌う
for s in C:
    if P[(1,s)] >= P[(2,s)]:
        prob += pulp.lpSum([x[(1,s,k)] for k in K]) >= pulp.lpSum([x[(2,s,k)] for k in K])
    else:
        prob += pulp.lpSum([x[(2,s,k)] for k in K]) >= pulp.lpSum([x[(1,s,k)] for k in K])
#7. 明はできるだけ連続
for k in K:
    if k != K[-1]:
        prob += y[k] >= pulp.lpSum(b[s]*[x[(i,s,k)] for i in I for s in S]) - pulp.lpSum([b[s]*x[(i,s,k+1)] for i in I for s in S])
        prob += y[k] >= pulp.lpSum([b[s]*x[(i,s,k+1)] for i in I for s in S]) - pulp.lpSum([b[s]*x[(i,s,k)] for i in I for s in S])
        prob += y[k] <= pulp.lpSum(b[s]*[x[(i,s,k)] for i in I for s in S]) + pulp.lpSum([b[s]*x[(i,s,k+1)] for i in I for s in S])
        prob += y[k] <= 2 - pulp.lpSum(b[s]*[x[(i,s,k)] for i in I for s in S]) - pulp.lpSum([b[s]*x[(i,s,k+1)] for i in I for s in S])

#8. 最初が明るい曲
prob += pulp.lpSum([b[s]*x[(i,s,1)] for i in I for s in S]) == 0
#9. 失恋ソング禁止
#10. で故っと局いっきょくはいれる


In [3]:
#求解
solve =prob.solve()
print(pulp.LpStatus[solve])
sumtime =0
sumc = 0
#結果の表示
for k in K:
    for i in I:
        for s in S:
            if x[(i,s,k)].value() == 1:
                sumtime += t[s]
                if s in C:
                    sumc += 1
                print(f"{i}:{b[s]}:{k}回目:{s}")
print(f"合計時間は{sumtime}分です")
print(f"共通の曲は{sumc}曲です")

Optimal
1:0:1回目:フライデーナイト
2:0:2回目:世界に一つだけの花
1:1:3回目:プレタポルテ
2:0:4回目:恋ダンス
1:1:5回目:涙そうそう
2:1:6回目:366日
1:1:7回目:春よ、来い
2:1:8回目:君が代
1:1:9回目:上を向いて歩こう
2:0:10回目:恋するフォーチュンクッキー
1:1:11回目:サイレントマジョリティー
2:1:12回目:GReeeeN
1:1:13回目:シャイニング・オン君が哀しい
2:0:14回目:プライバシー
1:0:15回目:ツバサをください
合計時間は59.5分です
共通の曲は10曲です
